In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
from datetime import datetime

In [ ]:
sc = SparkContext(appName="test4")

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('D:/Nisum/Employees.csv')
#df.show(5)

In [ ]:
"""
key_field--> Primary Key filed from dataframe
date_filed--> Date file from dataframe
current_dateFormat--> format of the existing date_filed which is coming from dataframe
custom_dateFromat--> format of date to change
"""

def customDateFormat(data_frame, key_field, date_field, current_dateFormat, custom_dateFormat):
    
    """converting columns into list
    here we are converting the dataframe values to list values for "key_field" and "date_field" 
    using rdd flat map function.those list values will be captured by Key_list and date_list."""
 
    key_list = data_frame.select(key_field).rdd.flatMap(lambda x: x).collect()
    date_list = data_frame.select(date_field).rdd.flatMap(lambda x: x).collect()
    
    """converting current date fromat into custom date format
    Here we are iterating the all date_list valuesto strptime function and changing the 
    current date format to custom date format using strftime function. converted list of value will be assiged to "dates"
    """
    
    dates = list(map(lambda x: datetime.strptime(x, current_dateFormat).strftime(custom_dateFormat), date_list))
    
    """Changing list  to dataframe
    Now we are converting both list values("key_list", "dates") to new datafarme using structtypes. dataframe name "key_date_df"
    """
    myschema= StructType([ StructField(key_field, StringType(), True),StructField(date_field, StringType(), True)])
    key_date_df=sqlContext.createDataFrame(zip(key_list,dates),schema = myschema)
    
    """
    Joining the 2 dataframes("key_date_df","data_frame")  using "key_field" and select required fields from both dataframes.
    """
    return data_frame.join(key_date_df, data_frame[key_field] == key_date_df[key_field],how='full').drop(data_frame[date_field]).drop(key_date_df[key_field]).select('*').sort(key_field).dropDuplicates()
     

In [ ]:
df2 = customDateFormat(df, "EMPLOYEE_ID", "HIRE_DATE", "%d-%m-%y", "%y")

In [ ]:
df2.show(5)